## Import library

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.optimizers as opt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPooling1D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  

In [2]:
def get_available_gpus():
    """
    code from http://stackoverflow.com/questions/38559755/how-to-get-current-available-gpus-in-tensorflow
    """
    from tensorflow.python.client import device_lib as _device_lib
    local_device_protos = _device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']

In [3]:
get_available_gpus()

['/device:CPU:0']

## Define path & total number

In [4]:
path = "./img/cat"
dirlist = os.listdir(path)
catPath = [os.path.join(path, dirname) for dirname in dirlist]
# path_noncat = os.path.join(path, 'no_cat')

In [5]:
total_img_num = sum(map(len,map(os.listdir, catPath)))
print('all kinds of cats: ', dirlist)
for i in range(len(catPath)):
    print(dirlist[i], ': ', len(os.listdir(catPath[i])))
print('total cat imgs: ', total_img_num)

all kinds of cats:  ['喜马拉雅猫', '奶牛猫', '孟买猫', '布偶猫', '扁脸加菲猫', '折耳猫', '暹罗猫', '橘猫', '玫瑰纹豹猫', '缅因猫', '美短', '芬克斯猫', '蓝猫', '金渐层', '银渐层']
喜马拉雅猫 :  700
奶牛猫 :  1043
孟买猫 :  1100
布偶猫 :  1113
扁脸加菲猫 :  1311
折耳猫 :  1084
暹罗猫 :  1260
橘猫 :  1380
玫瑰纹豹猫 :  780
缅因猫 :  1065
美短 :  863
芬克斯猫 :  911
蓝猫 :  1200
金渐层 :  1362
银渐层 :  1361
total cat imgs:  16533


## Train test split

In [6]:
batch_size = 128
epochs = 30
IMG_HEIGHT = 150
IMG_WIDTH = 150
split = 0.2
classNum = len(dirlist)
total_val = total_img_num * split
total_train = total_img_num - total_val

In [7]:
# Generator

train_datagen = ImageDataGenerator(
    rescale=1./255,rotation_range=40, # Angle, 0-180
    width_shift_range=0.2, # horizontal shifting
    height_shift_range=0.2, # vertical shifting
    shear_range=0.2, # Shearing
    zoom_range=0.2, # Zooming
    horizontal_flip=True, # Flipping
    validation_split=split
)

#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True,

In [8]:
# Load images from the disk, applies rescaling, and resizes the images

train_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation') # set as validation data

Found 13230 images belonging to 15 classes.
Found 3303 images belonging to 15 classes.


## Labels

In [9]:
import collections

def count_label(generator):
    l = len(generator)
    counter = collections.Counter()
    for i in range(l):
        _, label = generator[i]
        unique, counts = np.unique(label, return_counts=True)
        count_label = dict(zip(unique, counts))
        counter.update(count_label) 
    result = dict(counter)
    return result

In [10]:
# train_count = count_label(train_generator)
# val_count = count_label(validation_generator)

# print("Training labels:")
# print(train_count)
# print("Validation labels:")
# print(val_count)

OSError: cannot identify image file './img/cat\\暹罗猫\\880.jpg'

## Sample visualisation

In [0]:
sample_img_train, sample_label_train = next(train_generator)

In [0]:
def plotImg(img):
    fig, axes = plt.subplots(3, 5, figsize=(20, 20))
    axes = axes.flatten()
    for i, a in zip(img, axes):
        a.imshow(i)
        a.axis("off")
    plt.tight_layout()
    plt.show()

In [0]:
# plotImg(sample_img_train[:15])

## Model

In [0]:
VGGNet = Sequential([
    Conv2D(64, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(128, 3, padding='same', activation='relu'),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    # Conv2D(256, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    # Conv2D(512, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    # Conv2D(512, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(classNum, activation='softmax'),
])

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(32, 3, padding='same', activation='relu'),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(64, 3, padding='same', activation='relu'),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(classNum, activation='softmax')
])

In [0]:
# from tensorflow.keras.callbacks import LearningRateScheduler
# lr_base = 0.001
# epochs = 250
# lr_power = 0.9
# def lr_scheduler(epoch, mode='power_decay'):
#     '''if lr_dict.has_key(epoch):
#         lr = lr_dict[epoch]
#         print 'lr: %f' % lr'''
 
#     if mode is 'power_decay':
#         # original lr scheduler
#         lr = lr_base * ((1 - float(epoch) / epochs) ** lr_power)
#     if mode is 'exp_decay':
#         # exponential decay
#         lr = (float(lr_base) ** float(lr_power)) ** float(epoch + 1)
#     # adam default lr
#     if mode is 'adam':
#         lr = 0.001
 
#     if mode is 'progressive_drops':
#         # drops as progression proceeds, good for sgd
#         if epoch > 0.9 * epochs:
#             lr = 0.0001
#         elif epoch > 0.75 * epochs:
#             lr = 0.001
#         elif epoch > 0.5 * epochs:
#             lr = 0.01
#         else:
#             lr = 0.1
 
#     print('lr: %f' % lr)
#     return lr
 
# # 学习率调度器
# scheduler = LearningRateScheduler(lr_scheduler)

In [0]:
sgd = opt.SGD(lr=1e-1, decay=1e-6, momentum=0.8, nesterov=True)

In [0]:
model.compile(optimizer=sgd,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'],
             )

In [0]:
model.summary()

## Train model

In [0]:
# import tensorflow as tf
# import keras.backend.tensorflow_backend as KTF

# config = tf.ConfigProto()  
# config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
# session = tf.Session(config=config)

# # 设置session
# KTF.set_session(sess)

In [0]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
with tf.device('/gpu:0'):
    history = model.fit_generator(
        train_generator,
        steps_per_epoch = total_train // batch_size,
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps = total_val // batch_size     
    )

## Result visualisation 

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='lower left')
plt.title('Training and Validation Loss')

plt.show()